In [ ]:
# import the necessary packages:
import glob
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os

from keras.applications.mobilenet import MobileNet

In [ ]:
# initialize the initial learning rate, number of epochs to train for,and batch size:
INIT_LR = 1e-4
EPOCHS = 15
BS = 256

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
#data generator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
data_train = '/content/gdrive/MyDrive/MASK_DATASET'
train_generator = train_datagen.flow_from_directory(
        data_train,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        data_train,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
model.fit(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)
# construct the training image generator for data augmentation:

train_datagen =ImageDataGenerator(
                                  rescale=1./255,
                                  rotation_range=20,
	                                zoom_range=0.15,
	                                width_shift_range=0.2,
	                                height_shift_range=0.2,
	                                shear_range=0.15,
	                                horizontal_flip=True,
	                                fill_mode="nearest",
                                  validation_split=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255 ,validation_split=0.2)

In [ ]:
directory = '/content/gdrive/MyDrive/MASK_DATASET'
CATEGORIES=['with_mask','without_mask','incorrect_mask']
train = train_datagen.flow_from_directory(directory,
                                       target_size=(224, 224),
                                       color_mode="rgb",
                                       classes=CATEGORIES,
                                       class_mode="categorical",
                                       batch_size=256,
                                       shuffle=True,
                                       seed=42,
                                       save_to_dir=None,
                                       save_prefix="",
                                       save_format="png",
                                       follow_links=False,
                                       subset='training',
                                       interpolation="nearest",
                                      )

test = test_datagen.flow_from_directory(directory,
                                       target_size=(224, 224),
                                       color_mode="rgb",
                                       classes=CATEGORIES,
                                       class_mode="categorical",
                                       batch_size=256,
                                       shuffle=True,
                                       seed=42,
                                       save_to_dir=None,
                                       save_prefix="",
                                       save_format="png",
                                       follow_links=False,
                                       subset='validation',
                                       interpolation="nearest",
                                      )

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off:

baseModel = MobileNetV2(weights= 'imagenet', include_top=False,
  input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)


# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process

for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
callbacks_list = [  
    ModelCheckpoint('/content/gdrive/My Drive/service_weights.h5', 
                    monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),          
    EarlyStopping(monitor='val_accuracy',patience=10),
    ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001),
    #TensorBoard(log_dir=logdir, histogram_freq=1, write_graph=True, write_images=True, update_freq='epoch', profile_batch=2, embeddings_freq=1)

]

In [ ]:
# train the head of the network
print("[INFO] training head...")

H = model.fit_generator(
	                     train,
                       steps_per_epoch = train.samples // BS ,
	                     validation_data=test ,
                       validation_steps = test.samples // BS ,
	                     epochs=EPOCHS , 
                       callbacks=callbacks_list)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
# make predictions on the testing set

print("[INFO] evaluating network...")
predIdxs = model.predict(test, batch_size=BS)

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability:

predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report:
print(classification_report(test.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability:

predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report:
print(classification_report(test.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

In [ ]:
# plot the training loss and accuracy
N = 50
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")
files.download("plot.png")

In [3]:
!git init
!git config --global user.email "spidehetaati@gmail.com"
!git config --global user.name "sepidehetaati"

!git remote add origin https://github.com/sepidehetaati/real-time-mask-detection.git
!git branch -M main
!git push -u origin main



Initialized empty Git repository in /content/.git/
error: refname refs/heads/master not found
fatal: Branch rename failed
error: src refspec main does not match any.
error: failed to push some refs to 'https://github.com/sepidehetaati/real-time-mask-detection.git'


In [6]:
!git add /content/gdrive/MyDrive/train-mask.ipynb